In [88]:
import glob
import numpy as np
import cv2
import math


In [55]:
# set root path
root_origin = '/Users/scenery/Code/SomeGit/Stereo/Problems/stereo/left/'
root_corner = '/Users/scenery/Code/SomeGit/Stereo/Problems/stereo/corner/'
root_undistort = '/Users/scenery/Code/SomeGit/Stereo/Problems/stereo/undistort/'

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = sorted(glob.glob(root_origin + '*.jpg'))

In [57]:
for i, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        cv2.imwrite(root_corner + 'image%d.jpg' % i, img)
    
    if ret == False:
        print('%d image is false' % i)

8 image is false
9 image is false


In [58]:
objpoints[0].shape

(42, 3)

In [59]:
imgpoints[0].shape

(42, 1, 2)

In [110]:
imgpoints

[array([[[475.32184 , 264.62466 ]],
 
        [[440.50244 , 263.23285 ]],
 
        [[406.2218  , 261.7014  ]],
 
        [[372.6837  , 259.91016 ]],
 
        [[340.0107  , 258.24222 ]],
 
        [[308.49207 , 256.51584 ]],
 
        [[277.5959  , 255.09286 ]],
 
        [[476.69138 , 230.0038  ]],
 
        [[441.2503  , 228.63284 ]],
 
        [[406.59464 , 227.64816 ]],
 
        [[372.70612 , 226.32228 ]],
 
        [[339.55396 , 225.40205 ]],
 
        [[307.56766 , 224.25938 ]],
 
        [[276.92807 , 223.40599 ]],
 
        [[477.408   , 194.33427 ]],
 
        [[441.71268 , 193.62064 ]],
 
        [[406.76755 , 192.52245 ]],
 
        [[372.57828 , 192.05171 ]],
 
        [[339.26407 , 191.56076 ]],
 
        [[307.0833  , 191.06427 ]],
 
        [[275.86005 , 190.52164 ]],
 
        [[477.91467 , 158.32228 ]],
 
        [[442.11322 , 157.88603 ]],
 
        [[406.80106 , 157.49673 ]],
 
        [[372.3857  , 157.41675 ]],
 
        [[338.89185 , 157.39815 ]],
 
        [[30

In [111]:
objpoints

[array([[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [5., 0., 0.],
        [6., 0., 0.],
        [0., 1., 0.],
        [1., 1., 0.],
        [2., 1., 0.],
        [3., 1., 0.],
        [4., 1., 0.],
        [5., 1., 0.],
        [6., 1., 0.],
        [0., 2., 0.],
        [1., 2., 0.],
        [2., 2., 0.],
        [3., 2., 0.],
        [4., 2., 0.],
        [5., 2., 0.],
        [6., 2., 0.],
        [0., 3., 0.],
        [1., 3., 0.],
        [2., 3., 0.],
        [3., 3., 0.],
        [4., 3., 0.],
        [5., 3., 0.],
        [6., 3., 0.],
        [0., 4., 0.],
        [1., 4., 0.],
        [2., 4., 0.],
        [3., 4., 0.],
        [4., 4., 0.],
        [5., 4., 0.],
        [6., 4., 0.],
        [0., 5., 0.],
        [1., 5., 0.],
        [2., 5., 0.],
        [3., 5., 0.],
        [4., 5., 0.],
        [5., 5., 0.],
        [6., 5., 0.]]), array([[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [

In [112]:
len(objpoints)

22

In [113]:
# add homogeneous
a = objpoints[0]
a = np.c_[a, np.ones(a.shape[0])]
objpoints[0] = a

In [114]:
# calculate the homogeneous and delete it
objpoints[0] = np.apply_along_axis(lambda x:x/x[3], 1, objpoints[0])
objpoints[0] = np.delete(objpoints[0],3,axis=1)

In [95]:
# normalize and denormalize matrix
a = objpoints[0]
x = a[:, 0]
y = a[:, 1]
x_new = x.mean()
y_new = y.mean()
s_x = math.sqrt(2 / (x.var()) ** 2)
s_y = math.sqrt(2 / (y.var()) ** 2)
N_x = np.array([[s_x, 0. , -s_x * x_new],
                [0. , s_y, -s_y * y_new],
                [0. , 0. , 1.          ]])
N_x_inv = np.linalg.inv(N_x)

In [130]:
a = np.zeros((1,9))
b = np.ones((1,9))
c = np.append(a, b)
d = np.reshape(c, (2,9))
d[1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [137]:
M = np.zeros((42 * 2, 9))
a = objpoints[0]
b = imgpoints[0]
b = b.reshape(42, 2)
for i in range(a.shape[0]):
    X, Y, u, v = a[i][0], a[i][1], b[i][0], b[i][1]
    M[2 * i]     = np.array([-X, -Y, -1., 0., 0., 0., u * X, u * Y, u])
    M[2 * i + 1] = np.array([0., 0., 0., -X, -Y, -1., v * X, v * Y, v])

In [138]:
M

array([[-0.00000000e+00, -0.00000000e+00, -1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  4.75321838e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.64624664e+02],
       [-1.00000000e+00, -0.00000000e+00, -1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.40502441e+02,  0.00000000e+00,  4.40502441e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -1.00000000e+00, -0.00000000e+00, -1.00000000e+00,
         2.63232849e+02,  0.00000000e+00,  2.63232849e+02],
       [-2.00000000e+00, -0.00000000e+00, -1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.12443604e+02,  0.00000000e+00,  4.06221802e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.00000000e+00, -0.00000000e+00, -1.000000